#Thermal

In [ ]:
import numpy as np
import os
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score
from google.colab import files, drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load the model
model_path = '/content/drive/MyDrive/Models/vgg_Thermalmodel.h5'
model = load_model(model_path)
print("Model loaded successfully from Drive.")

Model loaded successfully from Drive.


In [ ]:
# load test zip file
print("Upload zip files")
uploaded = files.upload()

Upload zip files


Saving ir_data.zip to ir_data.zip


In [ ]:
import zipfile
image_folder = ''
for fname in uploaded.keys():
    # loads test images
    if fname.endswith('.zip'):
      image_folder = os.path.splitext(fname)[0]
      image_folder = os.path.join('/content', image_folder)
      os.makedirs(image_folder, exist_ok=True)
      with zipfile.ZipFile(fname, 'r') as zip_ref:
        zip_ref.extractall('/content')
      print("Images extracted")

Images extracted


In [ ]:
print(os.listdir(image_folder))
print(len(os.listdir(image_folder)))

['000121_ir.jpg', '000040_ir.jpg', '000044_ir.jpg', '000063_ir.jpg', '000109_ir.jpg', '000112_ir.jpg', '000058_ir.jpg', '000098_ir.jpg', '000126_ir.jpg', '000069_ir.jpg', '000055_ir.jpg', '000085_ir.jpg', '000047_ir.jpg', '000111_ir.jpg', '000006_ir.jpg', '000053_ir.jpg', '000078_ir.jpg', '000056_ir.jpg', '000088_ir.jpg', '000026_ir.jpg', '000114_ir.jpg', '000020_ir.jpg', '000041_ir.jpg', '000080_ir.jpg', '000037_ir.jpg', '000034_ir.jpg', '000072_ir.jpg', '000017_ir.jpg', '000100_ir.jpg', '000011_ir.jpg', '000027_ir.jpg', '000003_ir.png', '000018_ir.jpg', '000082_ir.jpg', '000031_ir.jpg', '000125_ir.jpg', '000068_ir.jpg', '000113_ir.jpg', '000070_ir.jpg', '000005_ir.jpg', '000107_ir.jpg', '000095_ir.jpg', '000124_ir.jpg', '000057_ir.jpg', '000071_ir.jpg', '000013_ir.jpg', '000059_ir.jpg', '000042_ir.jpg', '000102_ir.jpg', '000119_ir.jpg', '000007_ir.jpg', '000043_ir.jpg', '000116_ir.jpg', '000093_ir.jpg', '000019_ir.jpg', '000066_ir.jpg', '000074_ir.jpg', '000091_ir.jpg', '000033_ir.jp

In [ ]:
import tensorflow as tf
import pandas as pd
import os

image_csv = 'ir_image_names.csv'
image_names = pd.read_csv(image_csv, header=None)[0].tolist()
image_paths = [os.path.join(image_folder, name) for name in image_names]
dataset = tf.data.Dataset.from_tensor_slices(image_paths)

def process_image(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, [64, 64])
  #img = tf.cast(img, tf.float32) / 255.0
  return img

dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)
predictions = []

for images in dataset:
  preds = model.predict(images)
  preds = tf.argmax(preds, axis=1)
  predictions.extend(preds.numpy())

predictions_df = pd.DataFrame(predictions, columns=['predictions'])
predictions_df.to_csv('Laksita_ir.csv', index=False)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
